In [1]:
import pandas as pd 
import numpy as np 

In [2]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits= pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


# merging dataframes

In [5]:
movies=movies.merge(credits,on='title')

In [6]:
#columns we will keep
#genres
#id
#keywords
#title
#overview(summary)
#cast
#crew

In [7]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [8]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [9]:
movies.dropna(inplace=True)

In [10]:
movies.duplicated().sum()

0

In [11]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [12]:
type(movies.iloc[0].genres)

str

In [13]:
import ast

In [14]:
def convert(text):
    L=[]
    for i in ast.literal_eval(text): #to convert string to list
        L.append(i['name']) 
    return L 

In [15]:
convert(movies.iloc[0].genres)

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [16]:
movies['genres']=movies['genres'].apply(convert)

In [17]:
movies.iloc[0].keywords

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [18]:
movies['keywords']=movies['keywords'].apply(convert)

In [19]:
#taking top3 cast
def convert3(text):
    counter=0
    L=[]
    for i in ast.literal_eval(text): #to convert string to list
        if counter!=3:
            L.append(i['name']) 
            counter+=1
        else:
            break
    return L 

In [20]:
movies['cast']=movies['cast'].apply(convert3)

In [ ]:
movies.iloc[0].crew

In [22]:
def director(text):
    L=[]
    for i in ast.literal_eval(text): #to convert string to list
        if i['job']=='Director':
            L.append(i['name']) 
            break
    return L 

In [23]:
movies['crew']=movies['crew'].apply(director)

In [24]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

removing spaces #sam worthington to #samworthington
so that sam and worhtington dont become different entities

In [25]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [26]:
movies.head(2)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]


In [27]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [28]:
new_df=movies[['movie_id','title','tags']]

In [29]:
#convierting tags into strings
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\chand\AppData\Local\Temp\ipykernel_2892\965116114.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [30]:
#to lower case
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\chand\AppData\Local\Temp\ipykernel_2892\2961376010.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


love, loving, loved all are same words. so we need to convert them to 
love,love,love
therefore stemming

In [31]:
import nltk

In [32]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [33]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [34]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\chand\AppData\Local\Temp\ipykernel_2892\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [35]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [36]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [37]:
vectors[0]


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [ ]:
#the top 5000 common words
cv.get_feature_names()

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

similaroty is inverse of distance


In [40]:
similarity=cosine_similarity(vectors)

In [41]:
similarity[0]
#1st column is 1 as its the same movie.
#therefore the diagonal will be 1 throughout the matrix

array([1.        , 0.08346223, 0.0860309 , ..., 0.04499213, 0.        ,
       0.        ])

we need to sort the values in a descending order to get the most similar movies  
but we lose the index of the movies that way, so we use enumerate function


In [42]:
list(enumerate(similarity[0]))[:10]

[(0, 1.0000000000000002),
 (1, 0.08346223261119858),
 (2, 0.08603090020146065),
 (3, 0.0734718358370645),
 (4, 0.1892994097121204),
 (5, 0.10838874619051501),
 (6, 0.04024218182927669),
 (7, 0.14673479641335554),
 (8, 0.05923488777590923),
 (9, 0.0967301666813349)]

In [43]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(1216, 0.28676966733820225),
 (2409, 0.26901379342448517),
 (3730, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.25038669783359574)]

using key=lambda x:x[1] so the the sorting is dont on basis of the second factor and not the index  
[1:6] to give the top 5 similar movies

In [44]:
new_df[new_df['title']=="Batman Begins"].index[0]

119

In [45]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(new_df.iloc[i[0]].title)
        #print(i[0])
    
    

In [46]:
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [47]:
new_df.iloc[1216].title

'Aliens vs Predator: Requiem'

In [48]:
import pickle

In [49]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [50]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [51]:
pickle.dump(similarity,open('similarity.pkl','wb'))